# Serialise Detections to a CSV File

---

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow/supervision/blob/develop/docs/notebooks/detections-to-jsonsink.ipynb)

This cookbook introduce [sv.CSVSink](https://supervision.roboflow.com/develop/detection/tools/save_detections/#supervision.detection.tools.csv_sink.CSVSink) tool designed to write captured object detection data to file from video streams/file

Click the `Open in Colab` button to run the cookbook on Google Colab.

In [ ]:
!pip install -q inference requests tqdm supervision==0.19.0rc5

In [39]:
import supervision as sv
from supervision.assets import download_assets, VideoAssets
from inference import InferencePipeline
from inference.core.interfaces.camera.entities import VideoFrame
import numpy as np
import pandas as pd

In [ ]:
SOURCE_VIDEO_PATH = download_assets(VideoAssets.PEOPLE_WALKING)
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.7

## Initialize ByteTrack

In [41]:
byte_track = sv.ByteTrack()

## Initialize CSVSink and open sink

In [42]:
csv_sink = sv.CSVSink('detections.csv')
csv_sink.open()

## Process video to class to track detections and save detections to CSV file

All the operations we plan to perform for each frame of our video - detection, tracking, annotation, and write to csv - are encapsulated in a function named `callback`.

In [43]:
def callback(predictions: dict, frame: VideoFrame) -> np.ndarray:
    detections = sv.Detections.from_inference(predictions)
    detections = byte_track.update_with_detections(detections)
    csv_sink.append(detections, custom_data={'frame_number': frame.frame_id})
    print(f"Processed Frame ID: {frame.frame_id}")

In [ ]:
#Aliases: https://inference.roboflow.com/reference_pages/model_aliases/

REGISTERED_ALIASES = {
    "yolov8n-640": "coco/3",
    "yolov8n-1280": "coco/9",
    "yolov8m-640": "coco/8",
    "yolov8x-1280": "coco/10",
}

def resolve_roboflow_model_alias(model_id: str) -> str:
    return REGISTERED_ALIASES.get(model_id, model_id)

alias = "yolov8n-640"
model_name = resolve_roboflow_model_alias(alias)

pipeline = InferencePipeline.init(
    model_id=model_name,
    video_reference=SOURCE_VIDEO_PATH,
    on_prediction=callback,
    iou_threshold=IOU_THRESHOLD,
    confidence_threshold=CONFIDENCE_THRESHOLD,
)

In [ ]:
pipeline.start()
pipeline.join()

In [46]:
# Close CSVSink
csv_sink.close()

## Visualizate results of detections CSV data with Pandas

Let's take a look at our resulting data with by using Pandas.

It will also be created in your current directory with the name detections.csv as well.

In [47]:
df = pd.read_csv('detections.csv')
df

,x_min,y_min,x_max,y_max,class_id,confidence,tracker_id,frame_number
0,1460.000000,469.000000,1544.00000,641.00000,0,0.820143,180,1
1,1447.000000,320.000000,1516.00000,480.00000,0,0.791599,181,1
2,259.000000,435.000000,326.00000,610.00000,0,0.775027,182,1
3,1142.000000,950.000000,1245.00000,1080.00000,0,0.768063,183,1
4,665.000000,649.000000,744.00000,851.00000,0,0.762994,184,1
...,...,...,...,...,...,...,...,...
7917,1854.766600,259.925570,1914.51900,442.58704,0,0.715024,322,341
7918,391.996430,470.574280,460.88147,632.92770,0,0.506561,318,341
7919,74.250565,700.197940,173.04185,893.15936,0,0.477815,306,341
7920,1012.475770,170.831770,1058.31270,297.39798,0,0.565866,283,341
